In [ ]:
import pandas as pd
from FRA import *
from MSD import *
from random import choice, randint
from scipy.optimize import minimize, Bounds

In [2]:
focals, estrategias = create_regions_and_strategies(8)
data = pd.read_csv('../Data/humans_absent.csv')
data['Region'] = data['Region'].apply(lambda x: [int(y) for y in x[1:-1].split(',')])
data['RegionGo'] = data['RegionGo'].apply(lambda x: [int(y) for y in x[1:-1].split(',')])
data['Overlap'] = data['Overlap'].apply(lambda x: [int(y) for y in x[1:-1].split(' ')])
data['Sims1'] = data['Sims1'].apply(lambda x: [float(y) for y in x[1:-1].split(',')])
data.head()

,Absent1,Is_there,Dyad,Player,Round,Score,Region,RegionGo,Overlap,Sims1
0,Unicorn_Absent,Unicorn_Absent,140-615,PL1,1,23,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.671875, 0.0, 0.6666666666666666, 0.20967741..."
1,Unicorn_Absent,Unicorn_Absent,140-615,PL2,1,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.171875, 0.0, 0.16216216216216217, 0.1315789..."
2,Unicorn_Absent,Unicorn_Absent,140-615,PL1,9,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,Unicorn_Absent,Unicorn_Absent,140-615,PL2,9,27,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.421875, 0.0, 0.3409090909090909, 0.25531914..."
4,Unicorn_Absent,Unicorn_Absent,140-615,PL1,10,32,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
mean_sims = []
for i in range(8):
    data['S_' + nameRegion(i + 1)] = data['Sims1'].apply(lambda x: x[i])
    media = data['S_' + nameRegion(i + 1)].mean()
    print("Mean sim to " + nameRegion(i + 1), media)
    mean_sims.append(media)

mean_sims

In [ ]:
def get_strategy(sims):
    sims = [float(x) for x in sims]
    a = sims[0]
    b = sims[2]
    t = sims[3]
    l = sims[4]
    r = sims[5]
    A = int(np.round(64 * a, 0))
    B = int(b * (A + 32) / (1 + b))
    T = int(t * (A + 32) / (1 + t))
    L = int(l * (A + 32) / (1 + l))
    R = int(r * (A + 32) / (1 + r))
#    print(a, b, t, l, r)
#    print("A = ", A)
#    print("A int B =", B)
#    print("A int T =", T)
#    print("A int L =", L)
#    print("A int R =", int(r * (A + 32) / (1 + r)))
    Ma = min(T, L, 16, (16 + T + L - A))
    Mi = max(0, T - R, L - 16, T - 16)
#    print("Min(T, L, 16, (16 + T + L - A)) =", Ma)
#    print("Max(0, T - R, L - 16, T - 16) =", Mi)
    x = randint(Mi, Ma)
    y = T - x
    u = L - x
    v = A - x - y - u
#    print("x =", x)
#    print("y =", y)
#    print("u =", u)
#    print("v =", v)
    X = list(range(0,4)) + list(range(8,12)) + list(range(16,20)) + list(range(24, 28))
    Y = list(range(4,8)) + list(range(12,16)) + list(range(20,24)) + list(range(28, 32))
    U = list(range(32,36)) + list(range(40,44)) + list(range(48,52)) + list(range(56, 60))
    V = list(range(36,40)) + list(range(44,48)) + list(range(52,56)) + list(range(60, 64))
    X = list(np.random.choice(X, x, replace=False)) if x > 0 else []
    Y = list(np.random.choice(Y, y, replace=False)) if y > 0 else []
    U = list(np.random.choice(U, u, replace=False)) if u > 0 else []
    V = list(np.random.choice(V, v, replace=False)) if v > 0 else []    
    strategy = X + Y + U + V
#    print(strategy)
    return strategy

def shaky_hand(strategy, p=4):
    outs = np.random.choice(strategy, p) if len(strategy) > 0 else []
    complement = [i for i in range(64) if i not in strategy]
    ins = np.random.choice(complement, p) if len(complement) > 0 else []
    strategy = [i for i in strategy if i not in outs] + list(ins)
    return [i for i in strategy]

def err2_regions(region1, region2):
    sims1 = [sim_consist(region1, k) for k in focals]     
    sims2 = [sim_consist(region2, k) for k in focals]
    return sum([(sims1[i] - sims2[i])**2 for i in range(len(sims1))])

def err2_sim_region(sims1, region):
    sims2 = [sim_consist(region, k) for k in focals]
    print(sims2)
    return sum([(sims1[i] - sims2[i])**2 for i in range(len(sims1))])

def err2_sims(sims1, sims2):
    return sum([(sims1[i] - sims2[i])**2 for i in range(len(sims1))])

def err2_model(df, pl, modelParameters, Num_Loc, focals, estrategias, p):
    df['SimsPred'] = df.apply(lambda x: chooseStrategy(
                            x['Region'], x['Score'], x['Overlap'], 
                            pl, modelParameters, Num_Loc, focals, estrategias), 
                            axis = 1)
    df['SimsPred'] = df['SimsPred'].apply(lambda x: code2Vector(x, Num_Loc))
    df['SimsPred'] = df.apply(lambda x: err2_sim_region(x['Sims1'], x['SimsPred']), axis=1)
    return df['SimsPred'].sum()

def mean_strategy():
    mean_sims = [0.491, 0.079, 0.335, 0.322, 0.331, 0.328, 0.335, 0.276]
    mean_strategy = get_strategy(mean_sims)
    return shaky_hand(mean_strategy)
    
def mean_region():
    mean_sims = [0.491, 0.079, 0.335, 0.322, 0.331, 0.328, 0.335, 0.276]
    mean_strategy = get_strategy(mean_sims)
    strategy = shaky_hand(mean_strategy)
    return code2Vector(strategy, 8)

In [ ]:
region = mean_region()
imprime_region(region)

In [ ]:
region1 = mean_region()
imprime_region(region1)

In [ ]:
err2_regions(region, region1)

In [ ]:
data['SimsP'] = data['Sims1'].apply(lambda x: [sim_consist(mean_region(), k) for k in focals])
Baseline = data.apply(lambda x: err2_sims(x['Sims1'], x['SimsP']), axis=1).sum()
Baseline

In [ ]:
pl = 0
modelParameters = [0.1, 0.1, 0.05, 0.01, # Biases
                   1, 100, 31.5, # Win Stay
                   0, 100, 0.9, # Attraction
                   0, 100, 0.9 # Repulsion to complement
                  ]
Num_Loc = 8

n = 4
region = data.iloc[n]['Region']
overlap = data.iloc[n]['Overlap']
score = data.iloc[n]['Score']
print("Region from:")
imprime_region(region)
print("Score", score)
sims = [sim_consist(x, region) for x in focals]
print("Similarities:\n", ["%.3f" % v for v in sims])
attrs = attractiveness(region, score, overlap, pl, modelParameters, Num_Loc, focals, True)
strategy_selected = get_strategy(attrs)
#strategy_selected = shaky_hand(strategy_selected)
region1 = code2Vector(strategy_selected, 8)
print("Region estimated:")
imprime_region(region1)
regionGo = data.iloc[n]['RegionGo']
simsGo = data.iloc[n]['Sims1']
print("Region actually chosen:")
imprime_region(regionGo)
print("Estimation error:", err2_sim_region(simsGo, region1))

In [ ]:
modelParameters = [0, 0, 0, 0, # Biases
                   1, 100, 31.5, # Win Stay
                   1, 20, 0.9, # Attraction
                   1, 20, 0.7 # Repulsion to complement
                  ]
err2_model(data, 0, modelParameters, 8, focals, estrategias, 4)

In [ ]:
for p in range(6):
    print("Shaky hand value:", p, "     Error:", err2_model(data, 0, modelParameters, 8, focals, estrategias, p))

In [ ]:
def MSD(params, data, focals, estrategias):
    
    pl = 0
    Num_Loc = 8
    p = 4
    er = err2_model(data, pl, params, Num_Loc, focals, estrategias, p)
#    print("Parametros: " + str(params) + "\nError: " + str(er))    
    return er

def optimizar(parametros, minimos, maximos, PARS):
    x0 = np.array(parametros)
    bounds = Bounds(minimos, maximos)
    print("Finding fitting parameters. Please wait...")
    data = PARS[0]
    focals = PARS[1]
    estrategias = PARS[2]
    res = minimize(
        MSD, 
        x0, 
        args=(data, focals, estrategias), 
        method='trust-constr', 
        bounds=bounds, 
        options={'verbose':1}
    )
    return res

In [ ]:
modelParameters = [0.0005, 0.0005, 0.0005, 0.0005, # Biases
                   0, 0, 0, # Win Stay
                   0, 0, 0, # Attraction
                   0, 0, 0 # Repulsion to complement
                  ]
minimos = [0] * 13
maximos = [0.001, 0.001, 0.001, 0.001, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#maximos = [0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#maximos = [0.25, 0.25, 0.25, 0.25, 1, 100, 32, 1, 100, 1, 1, 100, 1]
PARS = [data, focals, estrategias]
res = optimizar(modelParameters, minimos, maximos, PARS)
print(res.x)
f = open('../Data/parameter_fit_humans.csv', 'w')
sl = ''.join(res.x)
f.close()

Finding fitting parameters. Please wait...


/usr/local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/usr/local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/usr/local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approx